# **Make english examples(json)**
make `english` examples to `eg/en .json`

### **Packages**

In [1]:
import json
import translatepy
import pandas as pd
import oxford_api
from oxford_api import Word
from multiprocessing.dummy import Pool as ThreadPool
from tqdm import tqdm

### **Functions**

In [2]:
#translator.example
'''
1. return a list of objects <class 'translatepy.translators.bing.Example'>
    - Note: it doent return list of strings. You need to convert to str() to use them.
2. return empty list [] if it doesnt have examples, but sometimes return None
'''
def expl_transpy(word):
    translator = translatepy.Translator()
    ex=translator.example(word,'es','en')
    if ex is None:
        ex = []
    ex = [str(x) for x in ex[:4]]
    translator.clean_cache()
    return ex

def get_examples(word):
    try:
        Word.get(word)
    except:
        expl = expl_transpy(word)[:4]
        if len(expl)==0:
            expl = None
        return expl
    else:
        expl=Word.examples()[:4]
        if len(expl)<4:
            expl = expl + expl_transpy(word)[:4]
            expl = expl[:4]
        if len(expl)==0:
            expl = None
        return expl

### **Read file merge.json**

In [4]:
df=pd.read_json('../merge.json')
df=df.sample(n=10)

##### **Create and delete `eg/en` folder**

In [1]:
#!rm -rf eg/en2

In [4]:
#!rm -rf eg/en
#!mkdir eg/en

### **Multiprocessing add english examples**

In [5]:
words=df['word'].tolist()[:5] #['position', 'ambitious', 'personal', 'only', 'tuesday']
words

['position', 'ambitious', 'personal', 'only', 'tuesday']

In [8]:
#words=['observer', 'balance', 'mate', 'conduct']
'''
i dont know what happen with this multiprocess, 
because return bad examples, correct implementacion is eg_en.ipynb
''''
#worker
def make_eg(word):
    data={}
    data['word']=word
    examples=get_examples(word)
    data['examples']=examples
    with open(f'eg/en2/{word}.json', 'w') as outfile:
        json.dump(data, outfile, indent=4)
    
    #pbar.update(1)

#multiprocessing
#pbar = tqdm(total=len(words))
pool = ThreadPool(100)
pool.starmap(make_eg, zip(words)) 
pool.close() 
pool.join()